In [1]:
from flask import Flask, request, jsonify
import os
from werkzeug.utils import secure_filename
import numpy as np
import cv2
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.models import Model
import joblib
import json
import requests

In [6]:
# --- Load Pre-trained Models and Data (as provided in your code) ---
# Load the pre-trained CNN model (MobileNetV2)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3), pooling='avg')
cnn_model = Model(inputs=base_model.input, outputs=base_model.output)

# Load the Random Forest model (assuming it's saved as 'rf_model.pkl')
try:
    # rf_model = joblib.load('D:\SoySoy\SoyBean\Models\rf_model.pkl')
    rf_model = joblib.load('D:\\SoySoy\\SoyBean\\Models\\rf_model.pkl')
except FileNotFoundError:
    print("Error: Random Forest model file 'rf_model.pkl' not found at the specified path.")
    rf_model = None

# Load saved features, labels, and class names
try:
    features = np.load('D:\SoySoy\SoyBean\Models\features.npy')
    labels = np.load('D:\SoySoy\SoyBean\Models\labels.npy')
except FileNotFoundError:
    print("Error: Features file 'features.npy' not found at the specified path.")
    features = None
    labels = None

try:
    with open('D:\SoySoy\SoyBean\Models\class_names.json', 'r') as f:
        class_names = json.load(f)
except FileNotFoundError:
    print("Error: Class names file 'class_names.json' not found at the specified path.")
    class_names = None

# --- Function to Extract Features from an Image using MobileNetV2 ---
def extract_feature_from_image(img_path):
    try:
        img = image.load_img(img_path, target_size=(224, 224))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_preprocessed = preprocess_input(img_array)
        features = cnn_model.predict(img_preprocessed)
        return features.flatten()  # Flatten the feature vector
    except Exception as e:
        print(f"Error extracting features from image: {e}")
        return None

# --- Function for Disease Severity Estimation using OpenCV ---
def estimate_severity(img_path):
    try:
        img = cv2.imread(img_path)
        img = cv2.resize(img, (224, 224))
        hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

        # Mask green leaf area
        lower_green = np.array([25, 40, 40])
        upper_green = np.array([90, 255, 255])
        leaf_mask = cv2.inRange(hsv, lower_green, upper_green)

        # Mask brown/yellowish (diseased area approx.)
        lower_brown = np.array([10, 50, 50])
        upper_brown = np.array([30, 255, 255])
        disease_mask = cv2.inRange(hsv, lower_brown, upper_brown)

        # Only count disease pixels within the leaf
        disease_in_leaf = cv2.bitwise_and(disease_mask, disease_mask, mask=leaf_mask)

        total_leaf = np.count_nonzero(leaf_mask)
        diseased_area = np.count_nonzero(disease_in_leaf)

        if total_leaf == 0:
            return 0

        severity = diseased_area / total_leaf
        print(f"🔬 Estimated Severity: {severity:.2%}")

        # Optional: Show mask results
        plt.subplot(1, 3, 1), plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB)), plt.title('Original')
        plt.subplot(1, 3, 2), plt.imshow(leaf_mask, cmap='gray'), plt.title('Leaf Mask')
        plt.subplot(1, 3, 3), plt.imshow(disease_in_leaf, cmap='hot'), plt.title('Disease Mask')
        plt.tight_layout()
        plt.show()

        return severity
    except Exception as e:
        print(f"Error estimating severity: {e}")
        return None

# --- Full Pipeline to Classify Disease and Estimate Severity ---
def classify_and_estimate_severity(img_path):
    if rf_model is None or class_names is None:
        print("Error: Pre-trained models or class names not loaded properly. Cannot classify disease.")
        return None, None, None

    # Step 1: Classify disease using CNN + RF
    features = extract_feature_from_image(img_path)
    if features is None:
        return None, None, None

    try:
        disease_pred = rf_model.predict([features])[0]
        disease_label = class_names[disease_pred]
    except Exception as e:
        print(f"Error during disease prediction: {e}")
        return None, None, None

    # Step 2: Estimate severity using image processing
    severity_score = estimate_severity(img_path)
    if severity_score is None:
        return disease_label, None, None

    severity_level = (
        "Mild" if severity_score < 0.2 else
        "Moderate" if severity_score < 0.5 else
        "Severe"
    )

    # Final Output
    print(f"🌿 Predicted Disease: {disease_label}")
    print(f"🩺 Estimated Severity: {severity_score:.2%} ({severity_level})")

    return disease_label, severity_level, severity_score

# --- Integration with OpenRouter API for Recommendations ---
# Replace with your OpenRouter API key
OPENROUTER_API_KEY = "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"

def get_openrouter_recommendation(disease_type, severity):
    api_url = "https://openrouter.ai/api/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {OPENROUTER_API_KEY}",
        "Content-Type": "application/json"
    }

    # You can experiment with different models available on OpenRouter
    # Check https://openrouter.ai/docs#models for a list
    model_name = "mistralai/mistral-medium"  # Example model

    prompt = f"""
    Provide detailed care and precaution recommendations for the soybean disease '{disease_type}' with '{severity}' severity, considering agricultural practices in Maharashtra, India.
    Consider factors like:
    - Immediate actions to take.
    - Preventative measures for the future.
    - Potential environmental considerations specific to the region.
    - Best practices for managing this specific disease and severity level.
    """

    payload = json.dumps({
        "model": model_name,
        "messages": [{"role": "user", "content": prompt}],
        "max_tokens": 300,  # Adjust as needed
        "temperature": 0.7  # Adjust for creativity vs. consistency
    })

    try:
        response = requests.post(api_url, headers=headers, data=payload)
        response.raise_for_status()  # Raise an exception for bad status codes
        response_json = response.json()
        if response_json and "choices" in response_json and len(response_json["choices"]) > 0:
            return response_json["choices"][0]["message"]["content"]
        else:
            return "Could not extract a recommendation from the OpenRouter response."
    except requests.exceptions.RequestException as e:
        return f"Error communicating with OpenRouter API: {e}"
    except json.JSONDecodeError:
        return "Error decoding JSON response from OpenRouter."

# --- Integrated Function to Get Classification, Severity, and Recommendations ---
def full_pipeline_with_recommendation(img_path):
    disease, severity_level, severity_score = classify_and_estimate_severity(img_path)

    if disease and severity_level:
        print("\n--- Recommendations from OpenRouter ---")
        recommendations = get_openrouter_recommendation(disease, severity_level)
        print(recommendations)
    elif disease:
        print("\n--- Recommendations cannot be generated as severity estimation failed. ---")
    else:
        print("\n--- Disease classification failed. Cannot provide recommendations. ---")

# --- Example Usage ---
if __name__ == "__main__":
    image_path = "D:\SoySoy\SoyBean\Test\septoria.jpeg"
    full_pipeline_with_recommendation(image_path)

ModuleNotFoundError: No module named 'numpy._core'

In [5]:
try:
    with open('D:\\SoySoy\\SoyBean\\Models\\rf_model.pkl', 'rb') as f:
        print("File opened successfully.")
except OSError as e:
    print(f"Error opening file: {e}")

File opened successfully.
